In [3]:
!pip install pandas numpy spacy sentence_transformers catboost scikit-learn
!python -m spacy download en_core_web_sm
!python -m spacy download ru_core_news_sm


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 80.3 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
  Using cached https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.7.0/ru_core_news_sm-3.7.0-py3-none-any.whl (15.3 MB)
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [39]:
DATA_PATH = '/kaggle/input/forhypee/users_with_target.parquet'  # Замените на путь к вашему датасету
user_new = pd.read_parquet(DATA_PATH)
user_new=user_new[~user_new["is_extravert"].isna()]

In [40]:
user_new["is_extravert"] = user_new["is_extravert"].astype(int)

In [37]:
user_new["is_extravert"] = user_new["is_extravert"].replace({
    'True': 1, 'False': 0,
    'yes': 1, 'no': 0
})

/tmp/ipykernel_30/331155792.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  user_new["is_extravert"] = user_new["is_extravert"].replace({


In [41]:
user_new

,vk_id,first_name,last_name,birth_date,age,user_id,avg_photo_likes,photo_count,post_count_owner,post_count_friend,...,diff_city_count,friend_count,groups_names,groups_descriptions,average_distance,cluster,posts_text,is_extravert,is_analyst,is_rigid
0,133343174,Анатолий,Голубец,1.1.2007,17.0,29,25.000000,4.0,0.0,1.0,...,-0.001,-30.0,"Рифмы и Панчи; NR; Берегите нервы, я мужчина-с...","РиП - новостной ресурс обо всём, что сейчас об...",0.952222,1.0,Уберите всю эту хуйню из своих голов.,1,False,False
1,164421917,Маша,Заверуха,1.1.2007,17.0,101,10.000000,5.0,2.0,0.0,...,-0.001,-30.0,Сороконожка-обувь.одежда.товары для дома,"Нашли такой же товар,но дешевле?Сообщите нам о...",0.932145,0.0,,0,False,True
2,476302530,Дина,Аверина,1.1.2007,17.0,154,3.500000,2.0,1.0,0.0,...,-0.001,-30.0,CYBERSHOKE; MegaRace; CS-ULET.RU - Игровой Про...,Открой CS2 по-новому на CYBERSHOKE.\n\n- 26 ре...,0.911009,1.0,[id476521218|Арина] открой лс,0,False,False
3,261985066,Матвей,Маркелов,2.1.2007,17.0,344,9.000000,3.0,1.0,19.0,...,-0.001,-30.0,женские форумы; Пиццерия Залупинск; Леонардо Д...,"Сообщество не является националистическим, не ...",0.931702,0.0,"Я вчера видел сон......; Teперь вы знаете, для...",0,False,False
4,499246599,Ilya,Savickij,2.1.2007,17.0,638,0.812500,16.0,19.0,1.0,...,-0.001,-30.0,Project Moon; подслушано project moon; Girls F...,18+\n\n• Новости и официальный контент от разр...,0.940610,1.0,"Дорогие Доктора, мы объявляем КОНКУРС!\n\nСпон...",1,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5045,7341858,Анна,Любомская,1.1.2008,16.0,18522,18.333333,9.0,0.0,20.0,...,31.000,125.0,У меня День Рождения в СЕНТЯБРЕ - by MaxShane ...,Здесь собрались все у кого день рождения в сен...,0.984739,0.0,"Срочно!!! \nТребуется работник в фаст-фуд, тол...",1,False,False
5046,2064914,Ансо,Юрлова,1.1.2008,16.0,18523,33.857143,7.0,5.0,15.0,...,33.000,236.0,Умный Новосибирск; ПОДЕЛКИ и РИСОВАНИЕ; Конный...,"Хотите, чтобы ваши дети осознанно подходили к ...",0.924235,1.0,Моя прекрасная лошадь в исполнении [id5675937|...,1,False,False
5047,196968673,Валюша,Одинец,1.1.2008,16.0,18524,3.444444,9.0,3.0,17.0,...,16.000,35.0,Удивительная планета; Россия Сегодня; Нетипичн...,Красивые и интересные места нашей удивительной...,0.934864,0.0,Привет из Белоруссии! Если вдруг в ближайшее в...,0,False,False
5048,20630619,Евгений,Лещенко,1.1.2008,16.0,18526,32.000000,19.0,20.0,0.0,...,16.000,113.0,YES Center - Образование и карьера за границей...,YES center - образование и стажировки за грани...,0.941110,1.0,Счастье уже 800 грамм#goodday#goodmood#may#yor...,1,False,False


In [26]:
user_new['is_extravert'].sum()

0.0

In [20]:
print(type(user_new['groups_names_emb'].iloc[0]))

<class 'str'>


In [42]:
import pandas as pd
import numpy as np
import torch
import re
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score
import spacy
from sentence_transformers import SentenceTransformer
from catboost import CatBoostClassifier, Pool

# 1. Загрузка языковых моделей Spacy
def load_spacy_models():
    try:
        nlp_en = spacy.load('en_core_web_sm')
    except OSError:
        print("Модель en_core_web_sm не найдена. Установите её с помощью команды:")
        print("python -m spacy download en_core_web_sm")
        raise

    try:
        nlp_ru = spacy.load('ru_core_news_sm')
    except OSError:
        print("Модель ru_core_news_sm не найдена. Установите её с помощью команды:")
        print("python -m spacy download ru_core_news_sm")
        raise

    return nlp_en, nlp_ru

# 2. Функция для очистки и лемматизации текста
def clean_and_lemmatize(text, nlp_en, nlp_ru, lang='ru'):
    """
    Очищает и лемматизирует текст.
    
    :param text: Исходный текст.
    :param nlp_en: Загруженная модель spaCy для английского.
    :param nlp_ru: Загруженная модель spaCy для русского.
    :param lang: Язык текста ('en' для английского, 'ru' для русского).
    :return: Лемматизированный текст.
    """
    if pd.isna(text):
        return ""
    
    # Обрезка до 100 символов
    text = text[:100]
    
    # Удаление специальных символов и приведение к нижнему регистру
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    
    # Выбор модели в зависимости от языка
    if lang == 'en':
        doc = nlp_en(text)
    elif lang == 'ru':
        doc = nlp_ru(text)
    else:
        raise ValueError("Поддерживаемые языки: 'en' для английского и 'ru' для русского.")
    
    # Лемматизация и удаление стоп-слов
    lemmas = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    
    return ' '.join(lemmas)

# 3. Функция для генерации эмбеддингов
def generate_embeddings(text_series, model, nlp_en, nlp_ru, lang='ru', batch_size=64):
    """
    Генерирует эмбеддинги для серии текстовых данных.
    
    :param text_series: pandas Series с текстовыми данными.
    :param model: Модель SentenceTransformer.
    :param nlp_en: Загруженная модель spaCy для английского.
    :param nlp_ru: Загруженная модель spaCy для русского.
    :param lang: Язык текста ('en' для английского, 'ru' для русского).
    :param batch_size: Размер батча для генерации эмбеддингов.
    :return: numpy.ndarray с эмбеддингами.
    """
    cleaned_texts = text_series.apply(lambda x: clean_and_lemmatize(x, nlp_en, nlp_ru, lang))
    embeddings = model.encode(cleaned_texts.tolist(), batch_size=batch_size, show_progress_bar=True)
    return embeddings

# 4. Основной пайплайн
def main(skip_ready=False):
    # Параметры
    DATA_PATH = '/kaggle/input/forhypee/users_with_target.parquet'  # Замените на путь к вашему датасету
    PROCESSED_DATA_PATH = 'processed_data.pkl'  # Путь для сохранения обработанного DataFrame
    MODEL_SAVE_PATH = 'catboost_models'  # Папка для сохранения моделей
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    
    TARGETS = ['is_extravert', 'is_analyst', 'is_rigid']
    TEXT_COLUMNS = {
        'groups_names': 'ru',
        'groups_descriptions': 'ru',
        'posts_text': 'ru'  # Измените язык при необходимости
    }
    
    # 1. Загрузка данных
    print("Загрузка данных...")
    if not skip_ready:
        df = pd.read_parquet(DATA_PATH)
        df = df[~df["is_extravert"].isna()]
        print(f"Датасет загружен с размером {df.shape}")
        # 1.1 Кодирование целевых переменных в бинарный формат (0 и 1)
        for target in TARGETS:
            if True:
                df[target] = df[target].astype(int)
            elif df[target].dtype == 'object':
                # Преобразование строковых меток в 0 и 1
                df[target] = df[target].map({
                    'True': 1, 'False': 0,
                    'true': 1, 'false': 0,
                    'Yes': 1, 'No': 0
                })
            else:
                # Преобразование числовых меток в 0 и 1
                df[target] = df[target].astype(int)
            
            # Замена пропусков на 0 (или другое значение, если необходимо)
            if df[target].isna().any():
                print(f"Пропуски обнаружены в целевой переменной '{target}'. Заполнение нулями.")
                df[target] = df[target].fillna(0)
            
            # Проверка уникальных значений
            unique = df[target].unique()
            if set(unique) - {0, 1}:
                print(f"Внимание: целевая переменная '{target}' содержит неожиданные метки: {unique}")
                # Преобразование всех меток >0 в 1, остальные в 0
                df[target] = df[target].apply(lambda x: 1 if x > 0 else 0)
                unique = df[target].unique()
                print(f"После преобразования: {unique}")
        # 2. Загрузка языковых моделей
        print("Загрузка языковых моделей Spacy...")
        nlp_en, nlp_ru = load_spacy_models()
        
        # 3. Загрузка модели эмбеддингов
        print("Загрузка модели SentenceTransformer...")
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model = SentenceTransformer('sergeyzh/rubert-tiny-turbo', device=device)
        
        # 4. Генерация эмбеддингов для текстовых столбцов
        embedding_feature_names = []
        for col, lang in TEXT_COLUMNS.items():
            print(f"Генерация эмбеддингов для столбца '{col}'...")
            embeddings = generate_embeddings(df[col], model, nlp_en, nlp_ru, lang=lang)
            # Добавляем эмбеддинги как одну колонку с векторами
            df[col + '_emb'] = embeddings.tolist()
            embedding_feature_names.append(col + '_emb')
            print(f"Эмбеддинги для '{col}' добавлены как новая колонка '{col}_emb'.")
        
        # 5. Подготовка признак
    else:
        df = pd.read_csv("data_with_embeds.csv")
    
    # Удаляем исходные текстовые столбцы
    df = df.drop(columns=list(TEXT_COLUMNS.keys()))
    
    # Определяем числовые и категориальные столбцы
    embedding_feature_names = ['groups_names_emb', 'groups_descriptions_emb', 'posts_text_emb']
    embedding_features = ['groups_names_emb', 'groups_descriptions_emb', 'posts_text_emb']
    # embedding_feature_names = []
    # for col, lang in TEXT_COLUMNS.items():
    #     emb_dim = 312  # Предполагаем размер эмбеддинга, проверьте фактический размер
    #     embedding_feature_names.extend([f"{col}_emb_{i}" for i in range(emb_dim)])
    
    # Остальные числовые столбцы (предполагаем, что они числовые)
    exclude_columns = ['vk_id', 'user_id', 'first_name', 'last_name', 'birth_date'] + TARGETS
    # Определяем числовые колонки (исключая эмбеддинговые)
    numerical_columns = [col for col in df.columns if col not in exclude_columns and col not in embedding_features]
    
    # Проверка на наличие числовых колонок
    if numerical_columns:
        # Нормализация числовых признаков
        scaler = StandardScaler()
        df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
    else:
        scaler = None
        print("Нет числовых колонок для нормализации.")
    
    df.to_pickle(PROCESSED_DATA_PATH)
    print(f"Обработанный DataFrame сохранен в '{PROCESSED_DATA_PATH}'")

    # 6. Разделение данных на обучающую и тестовую выборки с стратификацией по первому целевому
    print("Разделение данных на обучающую и тестовую выборки...")
    # Читать из pickle, чтобы убедиться, что список сохранен корректно
    df = pd.read_pickle(PROCESSED_DATA_PATH)

    X = df.drop(columns=TARGETS+['vk_id', 'user_id', 'first_name', 'last_name', 'birth_date'])
    y_dict = {target: df[target] for target in TARGETS}
    
    # Чтобы обеспечить одинаковое разбиение для всех целей, используем общий индекс разбиения
    train_indices, test_indices = train_test_split(
        df.index,
        test_size=0.2,
        random_state=42,
        stratify=None  # Можно выбрать одну цель для стратификации, если они сбалансированы
    )
    
    X_train = X.loc[train_indices]
    X_test = X.loc[test_indices]
    
    y_train_dict = {target: y_dict[target].loc[train_indices] for target in TARGETS}
    y_test_dict = {target: y_dict[target].loc[test_indices] for target in TARGETS}
    for target in TARGETS:
        print(f"\nПроверка уникальных меток для целевой переменной '{target}'...")
        print(f"Уникальные метки в обучающей выборке: {y_train_dict[target].unique()}")
        print(f"Уникальные метки в тестовой выборке: {y_test_dict[target].unique()}")
        
        # Убедимся, что обе выборки содержат оба класса
        if set(y_train_dict[target].unique()) != {0, 1}:
            print(f"Ошибка: Обучающая выборка для '{target}' не содержит оба класса. Пропуск модели.")
            continue
        
        if set(y_test_dict[target].unique()) != {0, 1}:
            print(f"Ошибка: Тестовая выборка для '{target}' не содержит оба класса. Пропуск модели.")
            continue

    # 8. Обучение и оценка моделей CatBoost
    for target in TARGETS:
        print(f"\nОбучение модели для целевой переменной '{target}'...")
        
        # Проверка уникальных меток
        print(f"\nПроверка уникальных меток для целевой переменной '{target}'...")
        print(f"Уникальные метки в обучающей выборке: {y_train_dict[target].unique()}")
        print(f"Уникальные метки в тестовой выборке: {y_test_dict[target].unique()}")
        
        # Убедимся, что обе выборки содержат оба класса
        if set(y_train_dict[target].unique()) != {0, 1}:
            print(f"Ошибка: Обучающая выборка для '{target}' не содержит оба класса. Пропуск модели.")
            continue
        
        if set(y_test_dict[target].unique()) != {0, 1}:
            print(f"Ошибка: Тестовая выборка для '{target}' не содержит оба класса. Пропуск модели.")
            continue
        
        # Подготовка Pool с указанием эмбеддинг признаков
        train_pool = Pool(
            X_train,
            y_train_dict[target],
            embedding_features=embedding_feature_names
        )
    
        test_pool = Pool(
            X_test,
            y_test_dict[target],
            embedding_features=embedding_feature_names
        )
    
        # Обработка дисбаланса классов (опционально)
        class_counts = y_train_dict[target].value_counts().to_dict()
        total = sum(class_counts.values())
        class_weights = {cls: total / count for cls, count in class_counts.items()}
        print(f"Классовые веса для '{target}': {class_weights}")
    
        # Инициализация модели
        model_cb = CatBoostClassifier(
            iterations=1000,
            learning_rate=0.1,
            depth=6,
            eval_metric='F1',
            random_seed=42,
            task_type='GPU' if device == "cuda" else 'CPU',
            class_weights=class_weights,  # Добавление классовых весов
            verbose=100
        )
    
        # Обучение модели
        model_cb.fit(
            train_pool,
            eval_set=test_pool,
            early_stopping_rounds=50,
            use_best_model=True
        )
    
        # Предсказания и оценка
        y_pred = model_cb.predict(X_test)
        
        # Проверка уникальных предсказанных меток
        unique_preds = np.unique(y_pred)
        print(f"Уникальные предсказанные метки для '{target}': {unique_preds}")
        
        # Убедимся, что y_pred содержит только 0 и 1
        if set(unique_preds) - {0, 1}:
            print(f"Ошибка: Предсказания модели '{target}' содержат неожиданные метки: {unique_preds}. Пропуск оценки.")
            continue
        
        accuracy = accuracy_score(y_test_dict[target], y_pred)
        f1 = f1_score(y_test_dict[target], y_pred, average='binary')
        print(f"Модель для '{target}': Accuracy = {accuracy:.4f}, F1-score = {f1:.4f}")
        print("Отчет классификации:")
        print(classification_report(y_test_dict[target], y_pred))
    
        # Сохранение модели
        model_path = os.path.join(MODEL_SAVE_PATH, f"catboost_{target}.cbm")
        model_cb.save_model(model_path)
        print(f"Модель сохранена в '{model_path}'")

    # 9. Сохранение scaler для последующей предобработки новых данных
    if scaler is not None:
        scaler_path = os.path.join(MODEL_SAVE_PATH, "scaler.pkl")
        with open(scaler_path, 'wb') as f:
            pickle.dump(scaler, f)
        print(f"Скалер сохранен в '{scaler_path}'")
    
    # 10. Сохранение embedding_feature_names для последующей загрузки
    embedding_features_path = os.path.join(MODEL_SAVE_PATH, "embedding_features.pkl")
    with open(embedding_features_path, 'wb') as f:
        pickle.dump(embedding_feature_names, f)
    print(f"Список эмбеддинг признаков сохранен в '{embedding_features_path}'")
    
    print("\nПайплайн завершен успешно.")

# 10. Функция для загрузки и использования модели
def load_and_predict(new_data, target, models_path='catboost_models'):
    """
    Загружает модель CatBoost и использует её для предсказания.
    
    :param new_data: pandas DataFrame с новыми данными.
    :param target: Целевая переменная для предсказания.
    :param models_path: Путь к сохраненным моделям.
    :return: Предсказания модели.
    """
    # Загрузка необходимых файлов
    scaler_path = os.path.join(models_path, "scaler.pkl")
    embedding_features_path = os.path.join(models_path, "embedding_features.pkl")
    model_path = os.path.join(models_path, f"catboost_{target}.cbm")
    
    if os.path.exists(scaler_path):
        with open(scaler_path, 'rb') as f:
            scaler = pickle.load(f)
    else:
        scaler = None
        print("Скалер не найден. Пропуск нормализации числовых признаков.")
    
    with open(embedding_features_path, 'rb') as f:
        embedding_feature_names = pickle.load(f)
    
    # Загрузка модели
    model_cb = CatBoostClassifier()
    model_cb.load_model(model_path)
    
    # Предобработка новых данных
    # Здесь необходимо повторить шаги предобработки: очистка текста, генерация эмбеддингов и нормализация
    # Для примера предполагаем, что new_data уже содержит эмбеддинги и числовые признаки
    
    # Например, если нужно повторить обработку:
    # cleaned_texts = new_data['groups_names'].apply(lambda x: clean_and_lemmatize(x, nlp_en, nlp_ru, lang='ru'))
    # embeddings = model.encode(cleaned_texts.tolist())
    # new_data['groups_names_emb'] = embeddings.tolist()
    # и т.д. для других текстовых колонок
    
    # Нормализация числовых признаков
    numerical_columns = [col for col in new_data.columns if col not in embedding_feature_names and col not in ['vk_id', 'user_id', 'first_name', 'last_name', 'birth_date']]
    if scaler is not None and numerical_columns:
        new_data[numerical_columns] = scaler.transform(new_data[numerical_columns])
    
    # Создание Pool для предсказания
    prediction_pool = Pool(
        new_data,
        embedding_features=embedding_feature_names
    )
    
    # Предсказание
    predictions = model_cb.predict(prediction_pool)
    return predictions

if __name__ == "__main__":
    main()

Загрузка данных...
Датасет загружен с размером (5050, 23)
Загрузка языковых моделей Spacy...
Загрузка модели SentenceTransformer...
Генерация эмбеддингов для столбца 'groups_names'...


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

Эмбеддинги для 'groups_names' добавлены как новая колонка 'groups_names_emb'.
Генерация эмбеддингов для столбца 'groups_descriptions'...


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

Эмбеддинги для 'groups_descriptions' добавлены как новая колонка 'groups_descriptions_emb'.
Генерация эмбеддингов для столбца 'posts_text'...


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

Эмбеддинги для 'posts_text' добавлены как новая колонка 'posts_text_emb'.
Обработанный DataFrame сохранен в 'processed_data.pkl'
Разделение данных на обучающую и тестовую выборки...

Проверка уникальных меток для целевой переменной 'is_extravert'...
Уникальные метки в обучающей выборке: [1 0]
Уникальные метки в тестовой выборке: [0 1]

Проверка уникальных меток для целевой переменной 'is_analyst'...
Уникальные метки в обучающей выборке: [0 1]
Уникальные метки в тестовой выборке: [1 0]

Проверка уникальных меток для целевой переменной 'is_rigid'...
Уникальные метки в обучающей выборке: [0 1]
Уникальные метки в тестовой выборке: [1 0]

Обучение модели для целевой переменной 'is_extravert'...

Проверка уникальных меток для целевой переменной 'is_extravert'...
Уникальные метки в обучающей выборке: [1 0]
Уникальные метки в тестовой выборке: [0 1]
Классовые веса для 'is_extravert': {1: 1.7003367003367003, 0: 2.4278846153846154}
0:	learn: 0.7342168	test: 0.7324562	best: 0.7324562 (0)	total: 4